[![View notebook](https://img.shields.io/static/v1?label=render%20on&logo=github&color=87ce3e&message=GitHub)](https://github.com/open-atmos/PySDM/blob/main/examples/PySDM_examples/Shima_et_al_2009/fig_2.ipynb)
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/open-atmos/PySDM.git/main?urlpath=lab/tree/examples/PySDM_examples/Shima_et_al_2009/fig_2.ipynb)    
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/open-atmos/PySDM/blob/main/examples/PySDM_examples/Shima_et_al_2009/fig_2.ipynb)

#### based on Fig. 2 from Shima et al. 2009 (Q. J. R. Meteorol. Soc. 135)  "_The super‐droplet method for the numerical simulation of clouds and precipitation: a particle‐based and probabilistic microphysics model coupled with a non‐hydrostatic model_"   
https://doi.org/10.1002/qj.441

In [7]:
import sys
if 'google.colab' in sys.modules:
    !pip --quiet install open-atmos-jupyter-utils
    from open_atmos_jupyter_utils import pip_install_on_colab
    pip_install_on_colab('PySDM-examples')

In [8]:
from numpy import errstate
from IPython.display import display
from PySDM.backends import CPU, GPU
from PySDM_examples.Shima_et_al_2009.spectrum_plotter import SpectrumPlotter
from PySDM_examples.Shima_et_al_2009.settings import Settings
from PySDM_examples.Shima_et_al_2009.example import run
from PySDM_examples.utils import widgets

In [9]:
progbar = widgets.IntProgress(min=0, max=100, description='%')

In [10]:
def demo(**kwargs):
    with kwargs['freezer']:
        with errstate(all='raise'):
            settings = Settings(steps=[i * (kwargs['n_step'] // kwargs['n_plot']) for i in range(kwargs['n_plot'] + 1)])
            backend = GPU if kwargs['gpu'] else CPU
            settings.n_sd = 2 ** kwargs['n_SD']
            settings.adaptive = kwargs['adaptive']
            settings.dt = 10 if settings.adaptive else settings.dt
            states, _ = run(settings, backend, (widgets.ProgbarUpdater(progbar, settings.output_steps[-1]),))

        with errstate(invalid='ignore'):
            plotter = SpectrumPlotter(settings)
            plotter.smooth = kwargs['smooth']
            for step, state in states.items():
                plotter.plot(state, step * settings.dt)
            plotter.show()

In [11]:
n_SD = widgets.IntSlider(value=14, min=12, max=18, step=1, description='$log_2(n_{SD})$', continuous_update=False)
n_step = widgets.IntSlider(value=3600, step=100, min=100, max=3600, description='$n_{step}$', continuous_update=False)
n_plot = widgets.IntSlider(value=3, step=1, min=1, max=8, description='$n_{plot}$', continuous_update=False)
sliders = widgets.HBox([n_SD, n_step, n_plot])

adaptive = widgets.Checkbox(value=False, description='adaptive dt')
smooth = widgets.Checkbox(value=True, description='smooth plot')
gpu = widgets.Checkbox(value=False, description='GPU')
options = [adaptive, smooth]
if GPU.ENABLE:
    options.append(gpu)
boxes = widgets.HBox(options)
freezer = widgets.Freezer([n_SD, n_step, n_plot, adaptive, gpu])
self = widgets.interactive_output(demo, 
                                  {'freezer': freezer, 'n_SD': n_SD, 'n_step': n_step, 'n_plot': n_plot,
                                   'adaptive': adaptive, 'smooth': smooth, 'gpu': gpu})

display(sliders, boxes, progbar, self)

IntProgress(value=100, description='%')

Output()